Q1

In [82]:
import pandas as pd
from datetime import datetime

crsp_stocks = pd.read_csv('CRSP_Stocks.csv')

def calculate_stock_returns(crsp_stocks):
    # Computes the lagged stock market capitalization, value-weighted return and equal-weight return.
    #
    # Args:
    #   CRSP_Stocks:  PERMNO | Integer
    #                 date | Date
    #                 SHRCD | Integer
    #                 EXCHCD | Integer
    #                 RET | Factor
    #                 DLRET | Factor
    #                 PRC | Numeric
    #                 SHROUT | Integer
    #
    # Returns:
    #   Monthly_CRSP_Stocks:   Year | Integer
    #                          Month | Integer
    #                          Stock_lag_MV | Numeric
    #                          Stock_Ew_Ret | Numeric
    #                          Stock_Vw_Ret | Numeric

    # Filter out unwanted samples
    restricted_universe = crsp_stocks[(crsp_stocks['shrcd'].isin([10, 11])) & 
                                      (crsp_stocks['exchcd'].isin([1, 2, 3]))]

    # Clean data, first round
    cleaned_first_round = restricted_universe.copy()
    cleaned_first_round['date'] = pd.to_datetime(cleaned_first_round['date'], format='%Y-%m-%d')
    cleaned_first_round['adj_price'] = abs(cleaned_first_round['prc'])
    cleaned_first_round['market_cap'] = cleaned_first_round['adj_price'] * cleaned_first_round['shrout']
    cleaned_first_round['adj_return'] = pd.to_numeric(cleaned_first_round['ret'], errors='coerce')
    cleaned_first_round['adj_dlreturn'] = pd.to_numeric(cleaned_first_round['dlret'], errors='coerce')

    # Clean data, second round
    cleaned_second_round = cleaned_first_round[(cleaned_first_round['adj_price'] > 0) & 
                                                (cleaned_first_round['shrout'] > 0) & 
                                                (cleaned_first_round['adj_return'].notna() | 
                                                 cleaned_first_round['adj_dlreturn'].notna())].copy()
    cleaned_second_round['return'] = cleaned_second_round.apply(lambda row: 
                                                                (1 + row['adj_return']) * 
                                                                (1 + row['adj_dlreturn']) - 1 if 
                                                                row['adj_return'] and 
                                                                row['adj_dlreturn'] else 
                                                                row['adj_return'] or 
                                                                row['adj_dlreturn'], axis=1)
    cleaned_second_round['year'] = cleaned_second_round['date'].dt.year
    cleaned_second_round['month'] = cleaned_second_round['date'].dt.month
    cleaned_second_round['lag_market_cap'] = cleaned_second_round.groupby('permno')['market_cap'].shift()

    # Calculate stock returns and group into year, month
    cleaned_shifted_df = cleaned_second_round.loc[cleaned_second_round['lag_market_cap'].notna()].copy()
    cleaned_shifted_df['weighted_return'] = cleaned_shifted_df['return'] * cleaned_shifted_df['lag_market_cap']
    stock_lag_mv = cleaned_shifted_df.groupby(['year', 'month'])['lag_market_cap'].sum().reset_index()
    stock_vw_ret = cleaned_shifted_df.groupby(['year', 'month'])['weighted_return'].sum().reset_index()

    stock_ew_ret = cleaned_shifted_df.groupby(['year', 'month'])['return'].mean().reset_index()


    # Generate output
    result = pd.merge(stock_lag_mv, stock_vw_ret, on=['year', 'month'], how='outer')
    result = pd.merge(result, stock_ew_ret, on=['year', 'month'], how='outer')
    result['vw_return'] = result['weighted_return'] / result['lag_market_cap']
    output = result[['year', 'month', 'lag_market_cap', 'return', 'vw_return']].copy()
    output.columns = ['Year', 'Month', 'Stock_lag_MV', 'Stock_Ew_Ret', 'Stock_Vw_Ret']
    return output





In [83]:
print(crsp_stocks.columns)

Index(['permno', 'date', 'shrcd', 'exchcd', 'ret', 'dlret', 'prc', 'shrout'], dtype='object')


In [84]:
# Call the function and pass the DataFrame as an argument
output = calculate_stock_returns(crsp_stocks)

# Print the output DataFrame
print(output)


      Year  Month  Stock_lag_MV  Stock_Ew_Ret  Stock_Vw_Ret
0     1926      1  3.126156e+05           NaN  0.000000e+00
1     1926      2  2.678966e+07           NaN  0.000000e+00
2     1926      3  2.599648e+07      0.520000  5.000678e-08
3     1926      4  2.407832e+07           NaN  0.000000e+00
4     1926      5  2.523588e+07           NaN  0.000000e+00
...    ...    ...           ...           ...           ...
1159  2022      8  4.134389e+10      0.036550  2.424689e-08
1160  2022      9  3.970959e+10      0.012385  9.524498e-06
1161  2022     10  3.592743e+10      0.237963  8.755456e-07
1162  2022     11  3.872999e+10      0.022485  1.193272e-06
1163  2022     12  4.044670e+10           NaN  0.000000e+00

[1164 rows x 5 columns]


Q2

In [85]:
output = output.drop(index=[0, 1, 2, 3, 4, 5])
output

,Year,Month,Stock_lag_MV,Stock_Ew_Ret,Stock_Vw_Ret
6,1926,7,2.652423e+07,NaN,0.000000e+00
7,1926,8,2.751748e+07,NaN,0.000000e+00
8,1926,9,2.824204e+07,NaN,0.000000e+00
9,1926,10,2.821734e+07,NaN,0.000000e+00
10,1926,11,2.735173e+07,NaN,0.000000e+00
...,...,...,...,...,...
1159,2022,8,4.134389e+10,0.036550,2.424689e-08
1160,2022,9,3.970959e+10,0.012385,9.524498e-06
1161,2022,10,3.592743e+10,0.237963,8.755456e-07
1162,2022,11,3.872999e+10,0.022485,1.193272e-06


In [86]:
output["date"] = output["Year"] * 100 + output["Month"]
output

,Year,Month,Stock_lag_MV,Stock_Ew_Ret,Stock_Vw_Ret,date
6,1926,7,2.652423e+07,NaN,0.000000e+00,192607
7,1926,8,2.751748e+07,NaN,0.000000e+00,192608
8,1926,9,2.824204e+07,NaN,0.000000e+00,192609
9,1926,10,2.821734e+07,NaN,0.000000e+00,192610
10,1926,11,2.735173e+07,NaN,0.000000e+00,192611
...,...,...,...,...,...,...
1159,2022,8,4.134389e+10,0.036550,2.424689e-08,202208
1160,2022,9,3.970959e+10,0.012385,9.524498e-06,202209
1161,2022,10,3.592743e+10,0.237963,8.755456e-07,202210
1162,2022,11,3.872999e+10,0.022485,1.193272e-06,202211


In [96]:
ff_df=pd.read_csv('F-F_Research_Data_Factors.CSV',skiprows=3,nrows=1158)
ff_df.columns=['date','Mkt-RF', 'SMB', 'HML', 'RF']
merged_df = pd.merge(output,ff_df,on='date')
#Converting % to decimal
merged_df['Stock_Vw_Ret']=100*merged_df['Stock_Vw_Ret']
#Calculating Excess Return
merged_df["Estimated_Mkt-RF"]=merged_df['Stock_Vw_Ret']-merged_df['RF']
df = merged_df[['Estimated_Mkt-RF','Mkt-RF']]
annual_rf = merged_df['RF'].mean()*12
# annualized mean
annualized_mean = df.mean() * 12
# annualized standard deviation
annualized_std = df.std() * np.sqrt(12)
# annualized Sharpe Ratio
annualized_sharpe_ratio = (annualized_mean - annual_rf) / annualized_std
# skewness
skewness = df.skew()
# excess kurtosis
excess_kurtosis = df.kurtosis() - 3

rows = ['Annualized Mean', 'Annualized Standard Deviation', 'Annualized Sharpe Ratio', 'Skewness', 'Excess Kurtosis']
df_matrix = pd.DataFrame([annualized_mean, annualized_std, annualized_sharpe_ratio, skewness, excess_kurtosis], index=rows)
df_matrix.columns=['Estimated FF Market Excess Return',' Actual FF Market Excess Return']
df_matrix

,Estimated FF Market Excess Return,Actual FF Market Excess Return
Annualized Mean,-3.184184,8.012953
Annualized Standard Deviation,0.872366,18.544860
Annualized Sharpe Ratio,-7.312773,0.259787
Skewness,-1.163518,0.160264
Excess Kurtosis,-1.419259,4.442663


In [97]:
ff_df

,date,Mkt-RF,SMB,HML,RF
0,192607,2.96,-2.56,-2.43,0.22
1,192608,2.64,-1.17,3.82,0.25
2,192609,0.36,-1.40,0.13,0.23
3,192610,-3.24,-0.09,0.70,0.32
4,192611,2.53,-0.10,-0.51,0.31
...,...,...,...,...,...
1153,202208,-3.77,1.37,0.30,0.19
1154,202209,-9.35,-0.79,0.06,0.19
1155,202210,7.83,0.09,8.05,0.23
1156,202211,4.60,-3.40,1.38,0.29


In [98]:
# calculate correlation between 'Stock_Vw_Ret' and 'Mkt-RF'
corr = merged_df['Stock_Vw_Ret'].corr(merged_df['Mkt-RF'])

# calculate maximum absolute difference between 'Stock_Vw_Ret' and 'Mkt-RF'
max_diff = (merged_df['Stock_Vw_Ret'] - merged_df['Mkt-RF']).abs().max()

# print results
print(f"Correlation: {corr:.8f}")
print(f"Max absolute difference: {max_diff:.8f}")

Correlation: 0.11279615
Max absolute difference: 38.85000000
